<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/Dataset/Create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirement libraries

In [ ]:
!pip install mutmut
!pip install astor
!pip install libcst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 kB 12.3 MB/s eta 0:00:00
  Created wheel for mutmut: filename=mutmut-2.5.1-py2.py3-none-any.whl size=31324 sha256=510ba4125053c393f3bc25ad39d1c958c9471250ec70c5c1d7bac527d6d870ac
  Stored in directory: /root/.cache/pip/wheels/38/a3/c7/572da9a8ddf88686624a4f70d8605b887086a98f8a02b1ebec
Successfully built mutmut
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import csv
import json
from collections.abc import Iterator
import ast    # To convert code to AST
import astor  # To convert AST back to code
import random  # To select mutations randomly
import libcst as cst
import libcst.matchers as m

# Reading Dataset

In [ ]:
# Reading not completed dataset
path = '/content/drive/MyDrive/Bachelor\'s project/python_codes.csv'
df = pd.read_csv(path)
print(f"Number of Dataframe rows : {df.shape}")
df.head()

Number of Dataframe rows : (23187, 2)


,language,source
0,Python 3,"n, m = map(int, input().split())\na = []\nfor ..."
1,Python 3,"n, m = map(int, input().split())\na = []\nfor ..."
2,Python 3,#In the name of Allah\n\nfrom sys import stdin...
3,Python 3,"s = input()\nl, *v = (int(x) for x in input()...."
4,Python 3,"s = input()\nl, *v = (int(x) for x in input()...."


In [ ]:
# Droping language column
df = df.drop('language', axis=1)
print(df.shape)
df.head()

(23187, 1)


,source
0,"n, m = map(int, input().split())\na = []\nfor ..."
1,"n, m = map(int, input().split())\na = []\nfor ..."
2,#In the name of Allah\n\nfrom sys import stdin...
3,"s = input()\nl, *v = (int(x) for x in input()...."
4,"s = input()\nl, *v = (int(x) for x in input()...."


In [ ]:
# Removing codes cotain comments
def has_comments(code):
    return '#' in code

df = df[~df['source'].apply(has_comments)]
print(df.shape)
df = df.reset_index(drop=True)
df.head()

(17743, 1)


,source
0,"n, m = map(int, input().split())\na = []\nfor ..."
1,"n, m = map(int, input().split())\na = []\nfor ..."
2,"s = input()\nl, *v = (int(x) for x in input()...."
3,"s = input()\nl, *v = (int(x) for x in input()...."
4,"n = int(input())\na = list(map(int, input().sp..."


In [ ]:
# Reducing the number of rows
desired_rows = 3000
selected_list = []
final_df = pd.DataFrame(columns=['source'])
while desired_rows != len(selected_list):
  rand_row = random.randrange(0, df.shape[0])
  if rand_row not in selected_list:
    selected_list.append(rand_row)
    final_df = pd.concat([final_df, pd.DataFrame(df['source'][rand_row:rand_row+1])])
df = final_df.reset_index(drop=True)
print(df.shape)
df.head()

(3000, 1)


,source
0,"o = lambda: map(int, input().split())\nn,m = o..."
1,login = input()\nn = int(input())\nlogins = []...
2,"from collections import deque\n\nn, m, k = map..."
3,n = int(input())\ns = input()\nans = len(s)\ni...
4,"l, r, a = map(int, input().split())\n\ntotal =..."


# Mutanting code
In this section, we use mutation technique code using the CST library. Changing operations in the code and randomly replace with CST operations.
## Why we are using CST?
We are using CST instead of AST because we want to save the format of the structure too and CST does it for us.
<br>
In the output, we expect a mutant code and a line to tell us which line has changed.

In [ ]:
import ast
import random

def mutate_code(source_code):
    tree = ast.parse(source_code)

    mutation_type = random.choice(["value", "decision", "statement"])


    print(mutation_type)
    if mutation_type == "value":
        # Value Mutation: Change integer constants
        for node in ast.walk(tree):
            if isinstance(node, ast.Constant) and isinstance(node.value, int):
                node.value = random.randint(1, 10000)
                break  # Apply mutation to only one line

    elif mutation_type == "decision":
        # Decision Mutation: Change comparison operators
        for node in ast.walk(tree):
            if isinstance(node, ast.Compare):
                node.ops[0] = random.choice([ast.Gt(), ast.Lt(), ast.Eq(), ast.GtE(), ast.LtE()])
                break  # Apply mutation to only one line

    elif mutation_type == "statement":
        # Statement Mutation: Change the assigned variable name
        for node in ast.walk(tree):
            if isinstance(node, ast.Assign) and isinstance(node.targets[0], ast.Name):
                node.targets[0].id = random.choice(['x', 'y', 'z'])
                break  # Apply mutation to only one line

    # Convert the AST back into source code
    mutated_code = astor.to_source(tree).strip()
    mutated_lines = mutated_code.split('\n')

    # Print the mutated code
    # print(mutated_code)

    return ast.unparse(tree)

# Example code to mutate
source_code = '''
from decimal import *
import math
getcontext().prec=50


a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))'''

# Mutate the code
mutated_code = mutate_code(source_code)
print(mutated_code)


TypeError: CodeMutator.leave_Assign() takes 2 positional arguments but 3 were given

In [ ]:
# Change operation
import random
import libcst as cst

# Initialize the global variable
flag = False
op_list = []

class OperationChanger(cst.CSTTransformer):

    select_operations = [
        cst.Add, cst.Subtract, cst.Multiply, cst.Divide,
        cst.Modulo, cst.Power, cst.FloorDivide,
        cst.LeftShift, cst.RightShift,
        cst.BitOr, cst.BitXor, cst.BitAnd,
        cst.MatrixMultiply
    ]

    operations = [
        cst.Add(), cst.Subtract(), cst.Multiply(), cst.Divide(),
        cst.Modulo(), cst.Power(), cst.FloorDivide(),
        cst.LeftShift(), cst.RightShift(),
        cst.BitOr(), cst.BitXor(), cst.BitAnd(),
        cst.MatrixMultiply()
    ]

    def __init__(self):
        super().__init__()
        self.changed = False  # Flag to track if a change has been made
        global flag
        flag = False  # Ensure global flag is initialized properly

    def leave_BinaryOperation(self, original_node, updated_node):
        global flag  # Declare that we're working with the global 'flag'
        if not self.changed:

            # available_op_list = list(set(op_list).symmetric_difference(self.select_operations))
            available_op_list = list(set(self.select_operations) - set(op_list))
            if len(available_op_list) == 0:
              print("-----Available operations list is Empty-----")
              available_op_list = self.select_operations
              op_list.clear()

            current_op = random.choice(available_op_list)  # Get a random operation to check for
            # print(current_op)
            op_list.append(current_op)

            new_op_to_change = random.choice(self.operations)  # Pick a random new operation to replace
            if isinstance(updated_node.operator, current_op):  # Ensure the new operation is different
                self.changed = True  # Mark that a change has been made
                flag = True  # Set the global flag to True after making a change
                return updated_node.with_changes(
                    operator=new_op_to_change.with_changes(
                        whitespace_before=cst.SimpleWhitespace(value=''),
                        whitespace_after=cst.SimpleWhitespace(value='')
                    )
                )
        return updated_node

# Original code with multiple lines and parentheses
code = df['source'][2]
print(code)

# print(code)

while not flag:
  module = cst.parse_module(code)
  modified_tree = module.visit(OperationChanger())
  if not flag:
    print("No changes were made.")


print('\n')
c = modified_tree.code
print(c)

from collections import deque

n, m, k = map(int, input().split())
field = []
for i in range(n):
    field.append(input())
save = [deque() for i in range(n)]
for i in range(n):
    cur = [[0, 0, 0] for i in range(m)]
    last = -1
    for j in range(m - 1 , -1, -1):
        if (field[i][j] == "1"):
            if (last == -1):
                cur[j][1] = 1
            else:
                cur[j][1] = last - j
            last = j
    last = -1
    for j in range(m):
        if (field[i][j] == "1"):
            if (last == -1):
                cur[j][0] = 1
            else:
                cur[j][0] = j - last
            last = j
            cur[j][2] = j
            save[i].append(cur[j])
for cnt in range(k):
    mx = [-1, -1, -1]
    for i in range(n):
        if (len(save[i]) == 1):
            mx = max(mx, [1, i, 0])
        else:
            mx = max(mx, [save[i][0][1], i, 0], [save[i][-1][0], i, -1])
    if (mx[0] == -1):
        break
    else:
        if (mx[2] == 0):
       

In [73]:
import random
import libcst as cst
import libcst as cst
from decimal import Decimal
import math

# Initialize the global variable for operations
flag = False
op_list = []

class OperationChanger(cst.CSTTransformer):

    select_operations = [
        cst.Add, cst.Subtract, cst.Multiply, cst.Divide,
        cst.Modulo, cst.Power, cst.FloorDivide,
        cst.LeftShift, cst.RightShift,
        cst.BitOr, cst.BitXor, cst.BitAnd,
        cst.MatrixMultiply
    ]

    operations = [
        cst.Add(), cst.Subtract(), cst.Multiply(), cst.Divide(),
        cst.Modulo(), cst.Power(), cst.FloorDivide(),
        cst.LeftShift(), cst.RightShift(),
        cst.BitOr(), cst.BitXor(), cst.BitAnd(),
        cst.MatrixMultiply()
    ]

    def __init__(self):
        super().__init__()
        self.changed = False  # Flag to track if a change has been made
        global flag
        flag = False  # Ensure global flag is initialized properly

    def leave_BinaryOperation(self, original_node, updated_node):
        global flag  # Declare that we're working with the global 'flag'
        if not self.changed:

            # available_op_list = list(set(op_list).symmetric_difference(self.select_operations))
            available_op_list = list(set(self.select_operations) - set(op_list))
            if len(available_op_list) == 0:
              # print("-----Available operations list is Empty-----")
              available_op_list = self.select_operations
              op_list.clear()

            current_op = random.choice(available_op_list)  # Get a random operation to check for
            # print(current_op)
            op_list.append(current_op)

            new_op_to_change = random.choice(self.operations)  # Pick a random new operation to replace
            if isinstance(updated_node.operator, current_op):  # Ensure the new operation is different
                self.changed = True  # Mark that a change has been made
                flag = True  # Set the global flag to True after making a change
                return updated_node.with_changes(
                    operator=new_op_to_change.with_changes(
                        whitespace_before=cst.SimpleWhitespace(value=''),
                        whitespace_after=cst.SimpleWhitespace(value='')
                    )
                )
        return updated_node

# Define the NumberCollector and NumberModifier
class NumberCollector(cst.CSTVisitor):
    def __init__(self):
        self.numbers = []

    def visit_Integer(self, node):
        self.numbers.append(node)

    def visit_Float(self, node):
        self.numbers.append(node)

class NumberModifier(cst.CSTTransformer):
    def __init__(self, target_node):
        super().__init__()
        self.target_node = target_node

    def leave_Integer(self, original_node, updated_node):
        if original_node == self.target_node:
            return updated_node.with_changes(value=str(int(original_node.value) + 100))
        return updated_node

    def leave_Float(self, original_node, updated_node):
        if original_node == self.target_node:
            return updated_node.with_changes(value=str(float(original_node.value) - 100))
        return updated_node

# Define the ComparisonCollector and SingleComparisonModifier
class ComparisonCollector(cst.CSTVisitor):
    def __init__(self):
        self.comparisons = []

    def visit_Comparison(self, node):
        self.comparisons.append(node)

class SingleComparisonModifier(cst.CSTTransformer):
    def __init__(self, target_node):
        super().__init__()
        self.target_node = target_node

    def leave_Comparison(self, original_node, updated_node):
        if original_node == self.target_node:
            modified_ops = []
            for operator in updated_node.comparisons:
                # Modify the operator and remove whitespace before and after
                if isinstance(operator.operator, cst.LessThan):
                    modified_ops.append(
                        operator.with_changes(
                            operator=cst.GreaterThan().with_changes(
                                whitespace_before=cst.SimpleWhitespace(value=''),
                                whitespace_after=cst.SimpleWhitespace(value='')
                            )
                        )
                    )
                elif isinstance(operator.operator, cst.GreaterThan):
                    modified_ops.append(
                        operator.with_changes(
                            operator=cst.Equal().with_changes(
                                whitespace_before=cst.SimpleWhitespace(value=''),
                                whitespace_after=cst.SimpleWhitespace(value='')
                            )
                        )
                    )
                elif isinstance(operator.operator, cst.Equal):
                    modified_ops.append(
                        operator.with_changes(
                            operator=cst.LessThan().with_changes(
                                whitespace_before=cst.SimpleWhitespace(value=''),
                                whitespace_after=cst.SimpleWhitespace(value='')
                            )
                        )
                    )
                else:
                    modified_ops.append(operator)  # No change for other operators
            # Return the updated node with modified operators
            return updated_node.with_changes(comparisons=modified_ops)
        return updated_node

# Function to randomly select a section and apply the corresponding modification
def randomly_modify_code(tree):
    sections = ['operation'
    # , 'value', 'decision'
    ]
    selected_section = random.choice(sections)

    if selected_section == 'operation':
        print("Selected section: Operation")

        while not flag:
          transformer = OperationChanger()
          modified_tree = tree.visit(transformer)
        #   if not flag:
        #     print("No changes were made.")
        # print("Operation changed")
        return modified_tree

    elif selected_section == 'value':
        print("Selected section: Value")
        collector = NumberCollector()
        tree.visit(collector)
        if collector.numbers:
            target_node = random.choice(collector.numbers)
            transformer = NumberModifier(target_node)
            modified_tree = tree.visit(transformer)
            return modified_tree

    elif selected_section == 'decision':
        print("Selected section: Decision")
        collector = ComparisonCollector()
        tree.visit(collector)
        if collector.comparisons:
            target_node = random.choice(collector.comparisons)
            transformer = SingleComparisonModifier(target_node)
            modified_tree = tree.visit(transformer)
            return modified_tree

    return tree  # In case no modification was made


# Function to modify and print the code
def modify_and_print_code(code):
    # Parse the input code into a CST tree
    tree = cst.parse_module(code)

    # Apply random modifications
    modified_tree = randomly_modify_code(tree)

    # Convert the modified tree back to a code string
    modified_code = modified_tree.code

    # Print the modified code
    print("Original Code:")
    print(code)
    print("\nModified Code:")
    return modified_code

# Call the function

# Sample code
code = """
a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0.5:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))
"""

modified_code = modify_and_print_code(code)
print(modified_code)


Selected section: Operation
No changes were made.
-----Available operations list is Empty-----
No changes were made.
ok
Original Code:

a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0.5:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))


Modified Code:

a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0.5:
        cur=(a/b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))



In [74]:
# Find witch line has changed
changed_lines = []
line = 1;
for original, mutated in zip(code.split('\n'), modified_code.split('\n')):
    if original != mutated:
      print(f"Line {line} has changed")
      changed_lines.append(mutated)
    line += 1

# Output changed lines
if changed_lines:
    print("\n--- Changed Lines ---")
    for line in changed_lines:
        print(line)
else:
    print("No changes detected.")

Line 8 has changed

--- Changed Lines ---
        cur=(a/b)//2
